In [ ]:
'''
The approach leverages the strengths of both FAISS and fastdtw to efficiently handle the search and refinement process:

FAISS: Efficiently retrieves a large initial candidate pool based on the last step of the query sequence. This is fast and handles high-dimensional data well.
fastdtw: Iteratively refines the candidate pool by considering increasingly longer subsequences, making it feasible to handle long sequences without getting bogged down by the complexity of full DTW computations for all candidates.
This combination ensures that we maintain efficiency while still getting accurate results.

Summary of the Approach
Initial Candidate Retrieval: Use FAISS to quickly retrieve an initial set of candidates based on the last step of the query sequence.
Iterative Refinement: Use fastdtw to iteratively refine the candidate pool. Start with the last step of the query sequence and progressively consider longer subsequences.
At each step, compute the DTW distance for the current subsequence with the candidates.
Sort the candidates based on the DTW distance and remove the bottom 10%.
Continue until the candidate set is sufficiently small.
Final Selection: Once the candidate pool is sufficiently small, compute the full DTW distance for the remaining candidates and select the one with the smallest distance.
'''

import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import faiss

def create_index(vectors):
    d = vectors.shape[1]  # dimension of the vectors
    index = faiss.IndexFlatL2(d)  # L2 distance index
    index.add(vectors)
    return index

def faiss_search(index, query, k=1000):
    D, I = index.search(query, k)
    return I, D

def fastdtw_distance(seq1, seq2):
    distance, _ = fastdtw(seq1, seq2, dist=euclidean)
    return distance

def fastseqsearch(context_length, sequences, query_sequence, feature_dim):
    if not sequences:
        print("Dataset is empty.")
        return None

    flattened_sequences = np.vstack(sequences)  # Flatten the sequences for FAISS indexing
    index = create_index(flattened_sequences)

    # Step 1: Use the last step of the query sequence to find initial candidates
    last_step_query = query_sequence[-1].reshape(1, -1)
    initial_candidates, _ = faiss_search(index, last_step_query, k=1000)  # Start with 1000 candidates

    # Convert initial candidates to a set for efficient refinement
    candidate_set = set(initial_candidates.flatten() // context_length)  # Map back to sequence indices
    print(f"Initial candidate set size: {len(candidate_set)}")

    # Step 2: Iteratively refine the candidate list
    for i in range(2, len(query_sequence) + 1):
        subseq_query = query_sequence[-i:].reshape(-1, feature_dim)
        distances = []
        print(f"Subsequence length: {i}, Candidate set size: {len(candidate_set)}")

        for candidate_idx in candidate_set:
            try:
                candidate_seq = sequences[candidate_idx].reshape(context_length, feature_dim)
                distance = fastdtw_distance(subseq_query, candidate_seq[-i:])
                distances.append((candidate_idx, distance))
            except IndexError as e:
                print(f"IndexError for candidate_idx: {candidate_idx}, Max index: {len(sequences)-1}")
                continue

        # Sort distances and remove the bottom 10%
        distances.sort(key=lambda x: x[1])
        cutoff_index = max(1, len(distances) - len(distances) // 10)
        candidate_set = set([idx for idx, dist in distances[:cutoff_index]])
        print(f"New candidate set size after refinement: {len(candidate_set)}")

        if len(candidate_set) == 1:
            break

    # Step 3: Find the most similar sequence in the refined candidate list
    highest_match = None
    highest_score = float('inf')

    for candidate_idx in candidate_set:
        try:
            candidate_seq = sequences[candidate_idx].reshape(context_length, feature_dim)
            distance = fastdtw_distance(query_sequence, candidate_seq)
            if distance < highest_score:
                highest_score = distance
                highest_match = candidate_idx
        except IndexError as e:
            print(f"IndexError for candidate_idx: {candidate_idx}, Max index: {len(sequences)-1}")
            continue

    print(f"The most similar sequence is at index {highest_match} with a DTW distance of {highest_score}.")
    return highest_match

# Example usage
context_length = 10
sample = 1000
feature_dim = 256

# Generate Sample Data
sequences = [np.random.rand(context_length, feature_dim) for _ in range(sample)]
query_sequence = np.random.rand(context_length, feature_dim)  # Example query sequence

# Run the search
most_similar_index = fastseqsearch(context_length, sequences, query_sequence, feature_dim)
print(f"Most similar sequence index: {most_similar_index}")


In [ ]:
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import faiss

def create_index(vectors):
    """
    Creates a FAISS index from a given set of vectors.
    Args:
        vectors (numpy.ndarray): A numpy array of shape (nb, d) where 'nb' is the number of base vectors and 'd' is the dimension.
    Returns:
        faiss.IndexFlatL2: The created FAISS index.
    """
    d = vectors.shape[1]  # dimension of the vectors
    index = faiss.IndexFlatL2(d)  # L2 distance index
    index.add(vectors)
    return index

def faiss_search(index, query, k=1000):
    """
    Search for k nearest neighbors in the FAISS index.
    Args:
        index (faiss.IndexFlatL2): The FAISS index.
        query (numpy.ndarray): The query vector.
        k (int): Number of nearest neighbors to return.
    Returns:
        np.ndarray: Indices of the nearest neighbors.
        np.ndarray: Distances to the nearest neighbors.
    """
    D, I = index.search(query, k)
    return I, D

def fastdtw_distance(seq1, seq2):
    """
    Computes the DTW distance between two sequences using the fastdtw algorithm.
    Args:
        seq1 (numpy.ndarray): The first sequence.
        seq2 (numpy.ndarray): The second sequence.
    Returns:
        float: The DTW distance between the sequences.
    """
    distance, _ = fastdtw(seq1, seq2, dist=euclidean)
    return distance

def test_fastseqsearch(context_length, feature_dim, sequences, query_sequence, expected_result):
    """
    Test the FastSeqSearch algorithm with given sequences and a query sequence.
    Args:
        context_length (int): The length of each sequence.
        feature_dim (int): The dimensionality of each feature vector.
        sequences (list of numpy.ndarray): The list of sequences.
        query_sequence (numpy.ndarray): The query sequence.
        expected_result (int or None): The expected result index.
    """
    if not sequences:
        print("Dataset is empty.")
        assert expected_result is None, f"Test failed: expected {expected_result}, got None"
        return

    # Flatten the sequences for FAISS indexing
    flattened_sequences = np.vstack(sequences)
    index = create_index(flattened_sequences)

    # Use the last step of the query sequence to find initial candidates
    last_step_query = query_sequence[-1].reshape(1, -1)
    initial_candidates, _ = faiss_search(index, last_step_query, k=1000)

    # Map back to sequence indices
    candidate_set = set(initial_candidates.flatten() // context_length)
    print(f"Initial candidate set size: {len(candidate_set)}")

    # Iteratively refine the candidate list
    for i in range(2, len(query_sequence) + 1):
        subseq_query = query_sequence[-i:].reshape(-1, feature_dim)
        distances = []
        print(f"Subsequence length: {i}, Candidate set size: {len(candidate_set)}")

        for candidate_idx in candidate_set:
            try:
                candidate_seq = sequences[candidate_idx].reshape(context_length, feature_dim)
                distance = fastdtw_distance(subseq_query, candidate_seq[-i:])
                distances.append((candidate_idx, distance))
            except IndexError:
                continue

        # Sort distances and remove the bottom 10%
        distances.sort(key=lambda x: x[1])
        cutoff_index = max(1, len(distances) - len(distances) // 10)
        candidate_set = set([idx for idx, dist in distances[:cutoff_index]])
        print(f"New candidate set size after refinement: {len(candidate_set)}")

        if len(candidate_set) == 1:
            break

    # Find the most similar sequence in the refined candidate list
    highest_match = None
    highest_score = float('inf')

    for candidate_idx in candidate_set:
        try:
            candidate_seq = sequences[candidate_idx].reshape(context_length, feature_dim)
            distance = fastdtw_distance(query_sequence, candidate_seq)
            if distance < highest_score:
                highest_score = distance
                highest_match = candidate_idx
        except IndexError:
            continue

    print(f"Expected: {expected_result}, Got: {highest_match}")
    assert highest_match == expected_result, f"Test failed: expected {expected_result}, got {highest_match}"

# Fixed sequences for testing
context_length = 10
feature_dim = 256
fixed_sequences = [
    np.ones((context_length, feature_dim)),
    np.zeros((context_length, feature_dim)),
    np.full((context_length, feature_dim), 0.5),
    np.random.rand(context_length, feature_dim),
    np.random.rand(context_length, feature_dim)
]

# Test Cases
print("Running Base Case Tests:")
test_fastseqsearch(context_length, feature_dim, fixed_sequences, np.full((context_length, feature_dim), 0.5), 2)  # Query with an existing sequence

print("Running Edge Case Tests:")
# Empty dataset test
try:
    test_fastseqsearch(context_length, feature_dim, [], np.random.rand(context_length, feature_dim), None)
except AssertionError as e:
    print(e)

# Single sequence in dataset
test_fastseqsearch(context_length, feature_dim, [fixed_sequences[0]], fixed_sequences[0], 0)

# Large dataset
large_sequences = [np.random.rand(context_length, feature_dim) for _ in range(10000)]
large_sequences[5000] = np.full((context_length, feature_dim), 0.5)
test_fastseqsearch(context_length, feature_dim, large_sequences, np.full((context_length, feature_dim), 0.5), 5000)

# Long sequence test
long_sequences = [
    np.random.rand(100, feature_dim),
    np.full((100, feature_dim), 0.5),
    np.random.rand(100, feature_dim),
    np.random.rand(100, feature_dim),
    np.random.rand(100, feature_dim)
]
test_fastseqsearch(100, feature_dim, long_sequences, long_sequences[1], 1)
